### Notebook to test the approach of getting amenties from description

#### Please run all the cells to find the amenities via 2 methods : string search and ner method
##### given_desc_find_amenties_string_match : requires amenities dictionary present in the same repository and the descriptio itself
##### given_desc_find_amenties_ner : this requires model object present in the same repository and the description 

##### Note : NER method has not been explored and experimented completely 

In [11]:
import pandas as pd
import re
import spacy
import string
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [14]:
### Loading dataset [This could be changed to your own dataset]
df = pd.read_csv('assignment_hotel_training.csv')

In [3]:
### General processing of sentences : 
regex = re.compile('[%s]' % re.escape(string.punctuation))
def pre_processing_words(sent):
    ## Removing punctuations,extra spaces,striping ,lowering,removing stop words,lemmatize
    new_sent = regex.sub('', sent)
    new_sent = re.sub(' +',' ',new_sent).strip().lower()
    text_tokens = new_sent.split()
    tokens_without_sw = [lemmatizer.lemmatize(word) for word in text_tokens if not word in stopwords.words('english')]
    final_sent = ' '.join(tokens_without_sw)
    
    return final_sent

In [4]:
### Processing property description
def process_description(desc):
    all_desc = desc.split('|')
    all_desc = [re.sub(r'\.(?!\d)', '_',d).split('_') for d in all_desc]
    all_desc_sentences = [pre_processing_words(a).strip() for sublist in all_desc for a in sublist]
    all_desc_sentences = [i for i in all_desc_sentences if i != '']
    return all_desc_sentences

### Function to find amenities via NER approach

In [5]:
def given_desc_find_amenties_ner(description,nlp_model):
    description = process_description(description)
    
    final_amenities = []
    for desc in description:
        doc2 = nlp_model(desc)
        temp = []
        for ent in doc2.ents:
            temp.append(ent.text)
            
        final_amenities.append(temp)
        
    final_amenities = '|'.join(list(set([a for sublist in final_amenities for a in sublist])))
    
    return final_amenities

### Function to find amenities via string match approach

In [6]:
def given_desc_find_amenties_string_match(amenities_dict,description):
    description = pre_processing_words(description)
    amenities = []
    for amen in amenities_dict.keys():
        if amen in description:
            amenities.append(amenities_dict[amen])
    return amenities

### Finding amenities from amenities dictionary

In [13]:
with open('amenities_dictionary.pkl', 'rb') as f:
    amenities_dict = pickle.load(f)

'|'.join(given_desc_find_amenties_string_match(amenities_dict,df['property_description'][0]))


'Credit cards accepted|Airport Transfer|Hot & Cold water|Internet access|Doctor on Call|Television|Dry Cleaning|Phone|Telephone|Laundry|ac|WiFi|Laundry Service|WiFi Internet|Fi Internet|Cards|Housekeeping'

### NER Approach

In [9]:
nlp_model = spacy.load("output/model-last/") 
given_desc_find_amenties_ner(df['property_description'][0],nlp_model)

'laundry service|airport transfer|credit card accepted|telephone|television|dry cleaning|hot cold water|doctor call|wifi internet|housekeeping'